In [4]:
import cv2

def main(video_source, width=1280, height=720):
    # Inicializar la captura con el backend FFMPEG (recomendado para IP cams)
    cap = cv2.VideoCapture(video_source, cv2.CAP_FFMPEG)
    
    # Reducir el buffer al mínimo para evitar retrasos en la transmisión en vivo
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

    if not cap.isOpened():
        print(f"No se pudo conectar a la fuente: {video_source}")
        return

    window_name = "Cámara IP"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Conexión perdida o frame no recibido.")
                break

            # Redimensionar la imagen al tamaño deseado
            frame = cv2.resize(frame, (width, height))

            cv2.imshow(window_name, frame)

            # Salir con la tecla 'q' o ESC
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q') or key == 27:
                break
                
    except KeyboardInterrupt:
        pass
    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    # Asegúrate de que tu IP y puerto sean correctos
    url_camara = "http://192.168.1.44:8080/video"
    
    main(video_source=url_camara)